## Persiapan

In [77]:
# Import library
import json
import nltk
from nltk.stem import WordNetLemmatizer
import random
import string
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Dropout # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from nltk.corpus import stopwords
from textblob import TextBlob 

nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

# Inisialisasi variabel
documents = []
patterns = []  # Daftar pola kalimat
words = []  # Daftar semua kata unik yang ditemukan dalam patterns
classes = []  # Kata semua tag intent unik 

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sahru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
# JSON
# Konversi ke object python
with open("dataset.json", "r") as file:
    data = json.load(file)

print(data)

{'intents': [{'tag': 'perkenalan', 'patterns': ['hallo', 'hai', 'hi', 'hy', 'pagi', 'morning', 'siang', 'afternoon', 'sore', 'malam', 'bro', 'sis', 'kawan'], 'responses': ['Hai! Aku AI-chan. Mau tau informasi apa tentang Sahrul-sama?', 'Hallo! Aku AI-chan, salam kenal! Mau tau tentang informasi Sahrul-sama?']}, {'tag': 'salam', 'patterns': ['bye', 'good bye', 'selamat tinggal', 'sampai jumpa', 'see you', 'dadah'], 'responses': ['Bye! Semoga hari-harimu menyenangkan!', 'Dadahh! Senang bisa membantu, sampai jumpa!', 'Good bye! Sampai bertemu lagi!', 'Dahh, semoga harimu menyenangkan!', 'Sampai jumpa lagi! Semoga hari kamu luar biasa!']}, {'tag': 'salam2', 'patterns': ['terima kasih', 'makasih'], 'responses': ['Sama-sama', 'Wokee']}, {'tag': 'pujian', 'patterns': ['keren', 'hebat', 'wow', 'mantap', 'kamu hebat!', 'kamu keren!', 'super'], 'responses': ['Sahrul-sama memang hebat!', 'Sahrul-sama memang keren, ganteng juga!', 'Saya tidak hebat, hanya Pahlawan Himmel dan Sahrul-sama yang hebat

In [79]:
# Menghitung jumlah intents
jumlah_intents = len(data["intents"])

# Menampilkan jumlah intents
print("Jumlah intents:", jumlah_intents)

Jumlah intents: 30


## Pre Processing

In [80]:
# Spell Correction menggunakan TextBlob
def spell_correction(text):
    return str(TextBlob(text).correct())

In [81]:
for intent in data['intents']:  # Iterasi melalui array 'intents' dalam dataset
    for pattern in intent['patterns']:  # Iterasi melalui pola kalimat pada setiap intent
        # Koreksi ejaan menggunakan TextBlob
        corrected_pattern = spell_correction(pattern)
        
        # Tokenisasi setiap kata dalam pola kalimat yang telah dikoreksi
        w = nltk.word_tokenize(corrected_pattern)
        # Lemmatization dan filter kata umum menggunakan stopwords dari NLTK
        w = [lemmatizer.lemmatize(word.lower()) for word in w if word.lower() not in stop_words and word not in string.punctuation] 
        patterns.append(corrected_pattern)

        # Menambahkan dokumen (token) dan label intent (tag) ke dalam corpus
        documents.append((w, intent['tag']))

        # Menambahkan tag intent ke dalam daftar 'classes' jika belum ada
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [82]:
# Sortir kata-kata untuk konsistensi
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [83]:
# TF-IDF
# Inisialisasi TfidfVectorizer untuk TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(patterns)  # Menghasilkan vektor fitur TF-IDF

In [84]:
# Simpan words dan classes untuk digunakan dalam klasifikasi
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb')) 
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('class.pkl', 'wb'))

## Training

In [85]:
# Persiapkan variabel
training = []
output_empty = [0] * len(classes)

# Katikan setiap pola (kalimat) dengan tag (kelas) yang sesuai
for i, doc in enumerate(documents):
    # Membuat output untuk klasifikasi
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([X[i].toarray()[0], output_row]) 

In [86]:
# Acak fitur dan masukkan ke array
random.shuffle(training)
training = np.array(training, dtype=object)

# Membuat fitur dan target
train_x = list(training[:, 0])
train_y = list(training[:, 1])

## Build Model Menggunakan NN

In [87]:
model = Sequential()

model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

adam = Adam(learning_rate=0.001)  
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [88]:
# Latih model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0606 - loss: 3.4135
Epoch 2/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0541 - loss: 3.3799   
Epoch 3/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0659 - loss: 3.3441   
Epoch 4/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1397 - loss: 3.2714   
Epoch 5/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2005 - loss: 3.2460   
Epoch 6/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2515 - loss: 3.1631
Epoch 7/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1996 - loss: 3.1233
Epoch 8/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3134 - loss: 3.0104
Epoch 9/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2768 - loss: 2.9388
Epoch 10/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3081 - loss: 2.8576   
Epoch 11/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3220 - loss: 2.7199
Epoch 12/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/s

In [89]:
# Simpan model
model.save('model.h5')